In [1]:
import numpy as np
import arcode
import codec
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:

pref = './Paper_RB_14bit_COMPINTs/CI884/COMPINT'
nx=256
ny=480
ncomp=4


'''
pref = './Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=240
ncomp=4
'''

'''
pref = './Paper_RB_14bit_COMPINTs/CI32324/COMPINT'
nx=64
ny=120
ncomp=4
'''

"\npref = './Paper_RB_14bit_COMPINTs/CI32324/COMPINT'\nnx=64\nny=120\nncomp=4\n"

In [7]:
i_f = 3
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 256, 480, 4) , mean = 136.03014119466147 , std = 13.346571933553763 , Max = 214.0 , Min = 44.0 , type=float64
t_C_0 shape: (1, 256, 480) , mean = 115.9767333984375 , std = 4.452235365070101 , Max = 122.0 , Min = 44.0 , type=float64
t_C_1 shape: (1, 256, 480) , mean = 142.29813639322916 , std = 5.751094476080457 , Max = 214.0 , Min = 130.0 , type=float64
t_C_2 shape: (1, 256, 480) , mean = 146.55380045572917 , std = 8.345470486819782 , Max = 205.0 , Min = 135.0 , type=float64
t_C_3 shape: (1, 256, 480) , mean = 139.29189453125 , std = 5.2121202329003955 , Max = 186.0 , Min = 132.0 , type=float64


In [8]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (256, 480) , mean = 11.29189453125 , std = 5.212120232900396 , Max = 58 , Min = 4 , type=int8


In [9]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen122880
size1122892
size256602
ratio1= 1.00009765625
ratio2= 0.460628255208


In [10]:
fs = 1; fe = 4;
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        idx = i-fs
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
    
    print('\n')

1
  t_C shape: (1, 256, 480, 4) , mean = 140.19452107747395 , std = 19.08002197207108 , Max = 200.0 , Min = 72.0 , type=float64
 channel0 shape: (256, 480) , mean = -15.6963134765625 , std = 6.5970496347612295 , Max = -2 , Min = -56 , type=int8
   olen: 122880 Bytes
   size1: 122892 Bytes
   size2: 54317 Bytes
ratio1= 1.00009765625
ratio2= 0.442032877604
 channel1 shape: (256, 480) , mean = 20.880891927083333 , std = 9.547012552816339 , Max = 72 , Min = 0 , type=int8
   olen: 122880 Bytes
   size1: 122892 Bytes
   size2: 62744 Bytes
ratio1= 1.00009765625
ratio2= 0.510611979167
 channel2 shape: (256, 480) , mean = 23.457877604166665 , std = 13.192368638235616 , Max = 69 , Min = 0 , type=int8
   olen: 122880 Bytes
   size1: 122892 Bytes
   size2: 67982 Bytes
ratio1= 1.00009765625
ratio2= 0.553238932292
 channel3 shape: (256, 480) , mean = 20.135628255208335 , std = 10.208221042443643 , Max = 60 , Min = 3 , type=int8
   olen: 122880 Bytes
   size1: 122892 Bytes
   size2: 64809 Bytes
ratio

In [11]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.00009765625
   ratio_2= 0.5083251953125
 
2
   ratio_1= 1.00009765625
   ratio_2= 0.5040751139322917
 
3
   ratio_1= 1.00009765625
   ratio_2= 0.4690185546875
 
